In [1]:
import configparser
config = configparser.ConfigParser()

In [2]:
import os
cd = os.getcwd()
print(cd)

C:\Users\jacqu\OneDrive\Documents\MSDS-at-UVA-2023\DS5001\final_project_files


In [3]:
config.read("../final_project_files/env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [4]:
data_home, output_dir

('C:\\Users\\jacqu\\OneDrive\\Documents\\MSDS-at-UVA-2023\\DS5001\\final_project_files\\data',
 'C:\\Users\\jacqu\\OneDrive\\Documents\\MSDS-at-UVA-2023\\DS5001\\final_project_files\\output')

In [5]:
from glob import glob

In [6]:
txt_files = glob(f"{data_home}/*/*", recursive=True)
# glob("/Users/*/data/*.txt", recursive=True)

In [7]:
series_label = os.listdir(data_home)
print(series_label)

['.ipynb_checkpoints', 'ASOIAF', 'Dragon_Chronicles', 'Warrior_Cats']


In [8]:
import pandas as pd
import numpy as np

### ASOIAF: A Game of Thrones

In [9]:
print(txt_files[0])

C:\Users\jacqu\OneDrive\Documents\MSDS-at-UVA-2023\DS5001\final_project_files\data\ASOIAF\01_a_game_of_thrones.txt


In [10]:
print(txt_files[0].split('\\')[-2])

ASOIAF


In [11]:
pd.set_option('display.max_colwidth', None)

In [12]:
#books = pd.DataFrame()
#for each_doc in txt_files[:3]:
lines = open(txt_files[0], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"Table of Contents")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.iloc[135:145])

In [13]:
clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*HOUSE BARATHEON\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
print(text.loc[pat_a])
print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[0]
line_b = text.loc[pat_b].index[0] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

          line_str
line_num          
147       PROLOGUE
                 line_str
line_num                 
49196     HOUSE BARATHEON


In [14]:
text = text.loc[line_a : line_b]

In [15]:
# text.sample(10)

In [16]:
# text.head()

In [17]:
# text.tail()

In [18]:
chap_pat = r"^\s*[A-Z]+\s*$"
chap_lines = text.line_str.str.match(chap_pat, case=False)
# text.loc[chap_lines]

In [19]:
text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

In [20]:
text.chap_num = text.chap_num.ffill()
# text.sample(10)

In [21]:
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES.sample(10)

In [22]:
OHCO = ['series','book_title','chap_num', 'para_num', 'sent_num', 'token_num'] #index names

In [23]:
CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

In [24]:
para_pat = r'\n \n+'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[0].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [25]:
import nltk

In [26]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)

In [27]:
# SENTS.head()

In [28]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()

In [29]:
TOKENS.head()

token_str term_str
book_title           chap_num para_num sent_num token_num                   
01_a_game_of_thrones 1        0        1        1                We       we
                                                2            should   should
                                                3             start    start
                                                4              back     back
                                                5                 ,

In [30]:
asoiaf_1 = TOKENS

In [31]:
import re

### ASOIAF: A Clash of Kings

In [32]:
#books = pd.DataFrame()
#for each_doc in txt_files[:3]:
lines = open(txt_files[1], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
text.line_str = text.line_str.str.strip()
# text = text[~text.line_str.str.contains(r"Table of Contents")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.sample(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*APPENDIX—THE KINGS AND THEIR COURTS\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[0]
line_b = text.loc[pat_b].index[0] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
pattern = r'\b([1-9]|[1-9][0-9]|([1-5][0-9]{2})|600)\b'
def replace_thing(text):
    return re.sub(pattern, "", text)
text.line_str = text.line_str.apply(replace_thing)
# text.loc[32506]

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*"
chap_lines = text.line_str.str.match(chap_pat, case=False)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')

In [33]:
CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
pattern = r"\n \n+"
CHAPS.chap_str = CHAPS.chap_str.apply(replace_thing)
# CHAPS.head(1)

para_pat = r'\n'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[1].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)
PARAS.iloc[3:9]

SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
# SENTS.sample(2)

In [34]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title          chap_num para_num sent_num token_num                   
02_a_clash_of_kings 1        0        1        1               The      the
                                               2             comet    comet
                                               3                 ’         
                                               4                 s        s
                                               5              tail     tail

In [35]:
asoiaf_2 = TOKENS

### ASOIAF: A Storm of Swords

In [36]:
#books = pd.DataFrame()
#for each_doc in txt_files[:3]:
lines = open(txt_files[2], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
text.line_str = text.line_str.str.strip()
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.sample(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*Appendix\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[0]
line_b = text.loc[pat_b].index[0] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*([A-Z]+|Epilogue)\s*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES.head(10)

In [37]:
CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[2].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
#SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

In [38]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title           chap_num para_num sent_num token_num                   
03_a_storm_of_swords 1        0        1        1               The      the
                                                2               day      day
                                                3               was      was
                                                4              grey     grey
                                                5               and      and

In [39]:
asoiaf_3 = TOKENS

### ASOIAF: A Feast for Crows

In [40]:
lines = open(txt_files[3], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
# text.line_str = text.line_str.str.strip()
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

In [41]:
clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*MEANWHILE, BACK ON THE WALL...\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[0]
line_b = text.loc[pat_b].index[0] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*([A-Z\s]+|THE KRAKEN’S DAUGHTER)$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
LINES['line_str'] = LINES.line_str.str.replace(r"\”\n", "\"\n\n\n", regex=True)
# LINES.head(10)

In [42]:
CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\n+'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[3].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
# SENTS.sample(2)
# SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

In [43]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title           chap_num para_num sent_num token_num                   
04_a_feast_for_crows 1        0        1        1                 “         
                                                2           Dragons  dragons
                                                3                 ,         
                                                4                 ”         
                                                5              said     said

In [44]:
asoiaf_4 = TOKENS

### ASOIAF: A Dance with Dragons

In [45]:
lines = open(txt_files[4], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
# text.line_str = text.line_str.str.strip()
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

In [46]:
clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*WESTEROS\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[0]
line_b = text.loc[pat_b].index[0] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*([A-Z ]+|THE QUEEN’S HAND|EPILOGUE|THE MERCHANT’S MAN)$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
LINES['line_str'] = LINES.line_str.str.replace(r"\”\n", "\"\n\n\n", regex=True)
# LINES.head(10)

In [47]:
CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\n+'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[4].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
# SENTS.sample(2)
# SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

In [48]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  \
book_title              chap_num para_num sent_num token_num             
05_a_dance_with_dragons 1        0        1        1               The   
                                                   2             night   
                                                   3               was   
                                                   4              rank   
                                                   5              with   

                                                             term_str  
book_title              chap_num para_num sent_num token_num           
05_a_dance_with_dragons 1        0        1        1              the  
                                                   2            night  
                                                   3              was  
                                                   4             rank  
                                                   5             with

In [49]:
asoiaf_5 = TOKENS

### Warrior Cats: Into the Wild

In [50]:
import fitz

In [51]:
def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    document = fitz.open(pdf_path)

    # Initialize a list to hold each line of text
    lines = []

    # Loop through each page in the PDF
    for page in document:
        # Extract text from the page as blocks
        blocks = page.get_text("blocks")

        # Each block is a tuple where the fourth element is the text
        for block in blocks:
            # Split the text in each block into lines
            block_lines = block[4].split('\n')

            # Extend the lines list with the lines from this block
            lines.extend(block_lines)

    # Close the document
    document.close()

    # Create a DataFrame from the lines list
    df = pd.DataFrame(lines, columns=['line_str'])
    df.index.name = 'line_num'
    # Return the DataFrame
    return df

In [52]:
text = extract_text_from_pdf(txt_files[12])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"PMWarriorsintotheWild")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

In [53]:
text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[12].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [54]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
SENTS.sent_str.isna().sum()

0

In [55]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  term_str
book_title       chap_num para_num sent_num token_num                     
01_into_the_wild 1        0        1        1                  A         a
                                            2          half-moon  halfmoon
                                            3             glowed    glowed
                                            4                 on        on
                                            5             smooth    smooth

In [56]:
wc_1 = TOKENS

### Warrior Cats: Fire and Ice

In [57]:
text = extract_text_from_pdf(txt_files[13])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"WarriorsFireandIce")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

In [58]:
text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[13].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [59]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
SENTS.sent_str.isna().sum()

0

In [60]:
TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title      chap_num para_num sent_num token_num                   
02_fire_and_ice 1        0        1        1            Orange   orange
                                           2            flames   flames
                                           3            lapped   lapped
                                           4                at       at
                                           5               the      the

In [61]:
wc_2 = TOKENS

### Warrior Cats: Forest of Secrets

In [62]:
text = extract_text_from_pdf(txt_files[14])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"WarriorsForestofSecrets")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[14].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [63]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title           chap_num para_num sent_num token_num                   
03_forest_of_secrets 1        0        1        1              Cold     cold
                                                2           gripped  gripped
                                                3               the      the
                                                4            forest   forest
                                                5                 ,

In [64]:
wc_3 = TOKENS

### Warrior Cats: Rising Storm

In [65]:
text = extract_text_from_pdf(txt_files[15])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"WarriorsRisingStorm")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[15].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [66]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  term_str
book_title      chap_num para_num sent_num token_num                    
04_rising_storm 1        0        1        1                An        an
                                           2          agonized  agonized
                                           3             groan     groan
                                           4            echoed    echoed
                                           5            across    across

In [67]:
wc_4 = TOKENS

### Warrior Cats: A Dangerous Path

In [68]:
text = extract_text_from_pdf(txt_files[16])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"WarriorsaDangerousPath")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[16].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [69]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  \
book_title          chap_num para_num sent_num token_num                      
05_a_dangerous_path 1        0        1        1                     Inside   
                                               2                        the   
                                               3          kennel-that-moves   
                                               4                          ,   
                                               5                 everything   

                                                                 term_str  
book_title          chap_num para_num sent_num token_num                   
05_a_dangerous_path 1        0        1        1                   inside  
                                               2                      the  
                                               3          kennelthatmoves  
                                               4                           
                                               5               everything

In [70]:
wc_5 = TOKENS

### Warrior Cats: The Darkest Hour

In [71]:
text = extract_text_from_pdf(txt_files[17])
# text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"WarriorstheDarkestHour")]
text = text[~text.line_str.str.contains(r"6/26/2009")]
text = text[~text.line_str.str.contains(r"file:///H:/E-Books/E")]
# text = text[~text.line_str.str.match(r"\n")]
text['line_str'].replace('', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*PROLOGUE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b].tail(1))
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^\s*(PROLOGUE|CHAPTER)\s*\d*$"
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[17].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [72]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str  term_str
book_title          chap_num para_num sent_num token_num                    
06_the_darkest_hour 1        0        1        1              Rain      rain
                                               2              fell      fell
                                               3          steadily  steadily
                                               4                 ,          
                                               5          drumming  drumming

In [73]:
wc_6 = TOKENS

### Dragon Chronicles: The Fire Within

In [74]:
text = extract_text_from_pdf(txt_files[5])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
text = text[~text.line_str.str.contains(r"THE FIRE WITHIN")]
text = text[~text.line_str.str.contains(r"PART TWO")]
# text.dropna(subset=['line_str'], inplace=True)
# print(text.head(10))

clip_pats = [
    r"^\s*WELCOME TO WAYWARD CRESCENT\s*$",
    r"^\s*ABOUT THE AUTHOR\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a].index[-1])
# print(text.loc[pat_b].index[-1])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^[A-Z\!\"\’ \.\,\-\‘]+$"
a_list = ["DRAGONS’ DEN", "WELCOME TO SCRUBBLEY LIBRARY GARDENS", "SAFETY", "DO NOT ENTER", "KILNING IN PROGRESS"]
pattern = '|'.join([re.escape(word) for word in a_list])
text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
text.loc[8033, 'line_str'] = text.loc[8033, 'line_str'].replace(" ", "\n")
chap_lines = text.line_str.str.match(chap_pat)
# chap_lines
# text.loc[chap_lines].shape
# ["DRAGONS’ DEN", "WELCOME TO SCRUBBLEY LIBRARY GARDENS", "SAFETY", "DO NOT ENTER", "KILNING IN PROGRESS"]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[5].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [75]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title         chap_num para_num sent_num token_num                   
01_the_fire_within 1        0        1        1              Well     well
                                              2                 ,         
                                              3              here     here
                                              4                we       we
                                              5               are      are

In [76]:
dc_1 = TOKENS

### Dragon Chronicles: Icefire

In [77]:
text = extract_text_from_pdf(txt_files[6])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
# text.dropna(subset=['line_str'], inplace=True)
# text.head(10)

clip_pats = [
    r"^\s*THE WISHING DRAGON\s*$",
    r"^\s*ABOUT THE AUTHOR\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^[A-Z\!\"\’ \.\,\-\‘]+$"
# a_list = ["DRAGONS’ DEN", "WELCOME TO SCRUBBLEY LIBRARY GARDENS", "SAFETY", "DO NOT ENTER", "KILNING IN PROGRESS"]
# pattern = '|'.join([re.escape(word) for word in a_list])
# text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
# text.loc[8033, 'line_str'] = text.loc[8033, 'line_str'].replace(" ", "\n")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[6].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [78]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title chap_num para_num sent_num token_num                   
02_icefire 1        0        1        1             David    david
                                      2                 ,         
                                      3                if       if
                                      4              your     your
                                      5              face     face

In [79]:
dc_2 = TOKENS

### Dragon Chronicles: The Fire Eternal

In [80]:
text = extract_text_from_pdf(txt_files[7])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
text = text[~text.line_str.str.contains(r"Part Two")]
text = text[~text.line_str.str.contains(r"Part Three")]
# text.dropna(subset=['line_str'], inplace=True)
# text.head(10)

clip_pats = [
    r"^\s*ARCTIC ICE CAP, UNRECORDED TIME\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^([A-Z\!\"\’ \.\,\-\‘]+|Epilogue)$"
a_list = ["SVALBARD ARCHIPELAGO, UNRECORDED TIME", 
          "THE ISLAND ONCE CALLED THE TOOTH OF RAGNAR", 
          "GODDESS, SEDNA", 
          "KILNING IN PROGRESS"]
pattern = '|'.join([re.escape(word) for word in a_list])
text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
text.loc[2488, 'line_str'] = text.loc[2488, 'line_str'].replace("A", "a")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[7].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [81]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title          chap_num para_num sent_num token_num                   
03_the_fire_eternal 1        0        1        1                It       it
                                               2             began    began
                                               3              with     with
                                               4                 a        a
                                               5              wind     wind

In [82]:
dc_3 = TOKENS

### Dragon Chronicles: Fire Star

In [83]:
text = extract_text_from_pdf(txt_files[8])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
text = text[~text.line_str.str.contains(r"PART TWO")]
text = text[~text.line_str.str.contains(r"PART THREE")]
text = text[~text.line_str.str.contains(r"PART FOUR")]
text = text[~text.line_str.str.contains(r"EPILOGUE")]
# text.dropna(subset=['line_str'], inplace=True)
# text.head(10)

clip_pats = [
    r"^\s*1 THE SHOOTING\s*$",
    r"^\s*Acknowledgments\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] - 1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^[0-9]+ [A-Z0-9\!\"\’ \.\,\--\‘\…]+$"
a_list = ["42 Wayward Crescent, Scrubbley.", 
          "42 Wayward Crescent,"]
pattern = '|'.join([re.escape(word) for word in a_list])
text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
#text.loc[2488, 'line_str'] = text.loc[2488, 'line_str'].replace("A", "a")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines].shape

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[8].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [84]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title   chap_num para_num sent_num token_num                   
04_fire_star 1        0        1        1              Boom     boom
                                        2                 .         
                               2        1              Boom     boom
                                        2                 .         
                               3        1              Boom     boom

In [85]:
dc_4 = TOKENS

### Dragon Chronicles: Dark Fire

In [86]:
text = extract_text_from_pdf(txt_files[9])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
text = text[~text.line_str.str.contains(r"PART TWO")]
# text.dropna(subset=['line_str'], inplace=True)
# text.head(10)

clip_pats = [
    r"^\s*1 LUCY’S JOURNAL\s*$",
    r"^\s*Sometimes\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] 
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^[0-9]+ [A-Z0-9\!\"\’ \.\,\--\‘\…]+$"
#a_list = ["42 Wayward Crescent, Scrubbley.", 
#          "42 Wayward Crescent,"]
#pattern = '|'.join([re.escape(word) for word in a_list])
#text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
#text.loc[2488, 'line_str'] = text.loc[2488, 'line_str'].replace("A", "a")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[9].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [87]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title   chap_num para_num sent_num token_num                   
05_dark_fire 1        0        1        1                OK       ok
                                        2                 ,         
                                        3              this     this
                                        4                is       is
                                        5             weird    weird

In [88]:
dc_5 = TOKENS

### Dragon Chronicles: Fire World

In [89]:
text = extract_text_from_pdf(txt_files[10])
text['line_str'].replace('', '\n', inplace=True)
text['line_str'].replace(' ', '\n', inplace=True)
text = text[~text.line_str.str.contains(r"PART TWO")]
text = text[~text.line_str.str.contains(r"PART THREE")]
text = text[~text.line_str.str.contains(r"PART FOUR")]
text = text[~text.line_str.str.contains(r"PART FIVE")]
# text.dropna(subset=['line_str'], inplace=True)
# text.head(10)

clip_pats = [
    r"^\s*PART ONE\s*$",
    r"^\s*About the Author\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])
# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]
line_b = text.loc[pat_b].index[-1] 
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^[0-9]+\.\s*$"
#a_list = ["42 Wayward Crescent, Scrubbley.", 
#          "42 Wayward Crescent,"]
#pattern = '|'.join([re.escape(word) for word in a_list])
#text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
#text.loc[2488, 'line_str'] = text.loc[2488, 'line_str'].replace("A", "a")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[10].split('\\')[-1].split('.pdf')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [90]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str   term_str
book_title    chap_num para_num sent_num token_num                      
06_fire_world 1        0        1        1          Professor  professor
                                         2           Merriman   merriman
                                         3                  .           
                                2        1              Eliza      eliza
                                         2                  .

In [91]:
dc_6 = TOKENS

### Dragon Chronicles: The Fire Ascending

In [92]:
lines = open(txt_files[11], 'r', encoding='utf-8').readlines()
text = pd.DataFrame(lines)
text.columns = ['line_str']
text.index.name = 'line_num'
text.line_str = text.line_str.str.strip()
text = text[~text.line_str.str.contains(r"PART ONE")]
text = text[~text.line_str.str.contains(r"PART TWO")]
text = text[~text.line_str.str.contains(r"PART THREE")]
text = text[~text.line_str.str.contains(r"PART FOUR")]
text = text[~text.line_str.str.contains(r"PART FIVE")]
text = text[~text.line_str.str.contains(r"PART SIX")]
text = text[~text.line_str.str.contains(r"PART SEVEN")]
text = text[~text.line_str.str.contains(r"PART EIGHT")]
text = text[~text.line_str.str.contains(r"PART NINE")]
text = text[~text.line_str.str.contains(r"PART TEN")]
text['line_str'].replace('', '\n', inplace=True)

clip_pats = [
    r"^\s*Voss\s*$",
    r"^\s*THE END\s*$"
]
# finding where the pattern is
pat_a = text.line_str.str.match(clip_pats[0])
pat_b = text.line_str.str.match(clip_pats[1])
# print(text.loc[pat_a])
# print(text.loc[pat_b])

# finding the line where the pattern is
line_a = text.loc[pat_a].index[-1]-1
line_b = text.loc[pat_b].index[-1]-1
# text._get_value(line_a, 'line_str'), text._get_value(line_b+1, 'line_str')

text = text.loc[line_a : line_b]
# text.sample(10)

chap_pat = r"^([0-9]*\. [A-Z0-9:\!\’ \.\,\‘]+|Voss|[A-Z0-9\!\’ \.\,\‘]+)$"
a_list = ["REGION, UNRECORDED TIME", 
          "SCRUBBLEY",
         "OF GWILANNA",
         "LISTENING DRAGON",
         "OF A LIBRARY GARDEN",
         "GWILANNA",
         "DARKLINGS"]
pattern = '|'.join([re.escape(word) for word in a_list])
text.line_str = text.line_str.apply(replace_thing)
#pattern = r" "
#text.line_str = text.line_str.apply(replace_thing)
text.loc[4498, 'line_str'] = text.loc[4498, 'line_str'].replace("1", "one")
text.loc[8251, 'line_str'] = text.loc[8251, 'line_str'].replace("42.", "forty-two")
text.loc[11044, 'line_str'] = text.loc[11044, 'line_str'].replace("HRRR!", "hrrr!")
chap_lines = text.line_str.str.match(chap_pat)
# text.loc[chap_lines]

text.loc[chap_lines, 'chap_num'] = [i+1 for i in range(text.loc[chap_lines].shape[0])]
# text.loc[chap_lines].head()

text.chap_num = text.chap_num.ffill()
LINES = text.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines]
LINES.chap_num = LINES.chap_num.astype('int')
# LINES['line_str'] = LINES.line_str.str.replace(r"\.\n", ".\n\n\n", regex=True)
# LINES['line_str'] = LINES.line_str.str.replace(r"\” \n", "\"\n\n\n", regex=True)
# LINES.head(10)

CHAPS = LINES.groupby(OHCO[2:3]).line_str.apply(lambda x: ' '.join(x))\
                                                .to_frame('chap_str')
CHAPS.chap_str = CHAPS.chap_str.str.strip()
# CHAPS.head(1)

para_pat = r'\n\s'
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[2:4]
# PARAS['para_str'] = PARAS.para_str.str.replace('\n ', ' ')
# PARAS['series'] = txt_files[0].split('\\')[-2]
PARAS['book_title'] = txt_files[11].split('\\')[-1].split('.txt')[0]
PARAS = PARAS.reset_index().set_index(OHCO[1:4])
# PARAS.head(5)

In [93]:
SENTS = PARAS.para_str.apply(nltk.sent_tokenize).to_frame('sent_str').explode('sent_str').reset_index()
SENTS['sent_num'] = SENTS.groupby(OHCO[1:4]).cumcount() + 1
SENTS.set_index(OHCO[1:5], inplace=True)
SENTS.sample(2)
SENTS = SENTS[SENTS.sent_str.isna()==False]
# SENTS.sent_str.isna().sum()

TOKENS = SENTS.sent_str.apply(nltk.word_tokenize).to_frame('token_str').explode('token_str').reset_index()
TOKENS['token_num'] = TOKENS.groupby(OHCO[1:5]).cumcount() + 1
TOKENS.set_index(OHCO[1:6], inplace=True)
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
TOKENS.head()

token_str term_str
book_title            chap_num para_num sent_num token_num                   
07_the_fire_ascending 1        0        1        1                 I        i
                                                 2               was      was
                                                 3                 a        a
                                                 4               boy      boy
                                                 5                of       of

In [94]:
dc_7 = TOKENS

### Making the CORPUS

In [95]:
CORPUS = pd.concat([asoiaf_1, asoiaf_2, asoiaf_3,
                     asoiaf_4, asoiaf_5,
                     wc_1, wc_2, wc_3, wc_4, wc_5, wc_6,
                     dc_1, dc_2, dc_3, dc_4, dc_5, dc_6, dc_7], axis=0)
CORPUS["pos"] = [pos for (word, pos) in nltk.pos_tag(CORPUS.term_str)]
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS.sample(10)

token_str term_str  \
book_title           chap_num para_num sent_num token_num                      
04_a_feast_for_crows 43       125      1        7              into     into   
01_into_the_wild     17       76       1        5               the      the   
04_a_feast_for_crows 35       11       5        7               the      the   
03_a_storm_of_swords 67       151      1        10             Keep     keep   
01_a_game_of_thrones 43       21       2        16                a        a   
04_a_feast_for_crows 46       37       4        18              the      the   
02_a_clash_of_kings  52       63       1        6              from     from   
04_rising_storm      9        57       1        12              for      for   
02_a_clash_of_kings  67       48       2        2              what     what   
01_a_game_of_thrones 12       45       1        34              her      her   

                                                           pos pos_group  
book_title           chap_num para_num sent_num token_num                 
04_a_feast_for_crows 43       125      1        7           IN        IN  
01_into_the_wild     17       76       1        5           DT        DT  
04_a_feast_for_crows 35       11       5        7           DT        DT  
03_a_storm_of_swords 67       151      1        10          NN        NN  
01_a_game_of_thrones 43       21       2        16          DT        DT  
04_a_feast_for_crows 46       37       4        18          DT        DT  
02_a_clash_of_kings  52       63       1        6           IN        IN  
04_rising_storm      9        57       1        12          IN        IN  
02_a_clash_of_kings  67       48       2        2           WP        WP  
01_a_game_of_thrones 12       45       1        34         PRP        PR

In [96]:
CORPUS.shape

(2789558, 4)

### Making the LIB

In [97]:
series = {'ASOIAF':['01_a_game_of_thrones',
                    '02_a_clash_of_kings',
                    '03_a_storm_of_swords',
                    '04_a_feast_for_crows',
                    '05_a_dance_with_dragons'], 
           'Warrior_Cats':['the_lord_of_the_rings',
                           '01_into_the_wild',
                           '02_fire_and_ice',
                           '03_forest_of_secrets',
                           '04_rising_storm',
                           '05_a_dangerous_path',
                           '06_the_darkest_hour'],  
           'Last_Dragon_Chronicles': ['01_the_fire_within',
                                      '02_icefire',
                                      '03_the_fire_eternal',
                                      '04_fire_star',
                                      '05_dark_fire',
                                      '06_fire_world',
                                      '07_the_fire_ascending']}

authors = {'George R.R. Martin':['01_a_game_of_thrones',
                    '02_a_clash_of_kings',
                    '03_a_storm_of_swords',
                    '04_a_feast_for_crows',
                    '05_a_dance_with_dragons'], 
           'Kate Cary':['01_into_the_wild',
                   '02_fire_and_ice',
                   '04_rising_storm'],
           'Cherith  Baldry':['03_forest_of_secrets', 
                      '05_a_dangerous_path',
                      '06_the_darkest_hour'],  
           'Chris D\'Lacey': ['01_the_fire_within',
                              '02_icefire',
                              '03_the_fire_eternal',
                              '04_fire_star',
                              '05_dark_fire',
                              '06_fire_world',
                              '07_the_fire_ascending']}

years = {'01_a_game_of_thrones': 1996,
         '02_a_clash_of_kings': 1998,
         '03_a_storm_of_swords': 2000,
         '04_a_feast_for_crows': 2005,
         '05_a_dance_with_dragons': 2001, 
         '01_into_the_wild': 2003,
         '02_fire_and_ice': 2003,
         '03_forest_of_secrets': 2003,
         '04_rising_storm': 2004,
         '05_a_dangerous_path': 2004,
         '06_the_darkest_hour': 2004,
         '01_the_fire_within': 2001,
         '02_icefire': 2003,
         '03_the_fire_eternal': 2005,
         '04_fire_star': 2007,
         '05_dark_fire': 2009,
         '06_fire_world': 2011,
         '07_the_fire_ascending': 2013}

In [98]:
LIB = CORPUS.groupby(level=0).term_str.count().to_frame('book_length')
LIB['n_chaps'] = CORPUS.reset_index()[['book_title','chap_num']]\
    .drop_duplicates()\
    .groupby('book_title').chap_num.count()

In [99]:
# Loop through each key-value pair in the dictionary
for key, indices in series.items():
    # print(key, indices)
    mask = LIB.index.isin(indices)
    # print(mask)
    LIB.loc[mask, 'series'] = key
    
LIB['year'] = pd.NA
for key, indices in years.items():
    LIB.loc[key, 'year'] = indices

for key, indices in authors.items():
    # print(key, indices)
    mask = LIB.index.isin(indices)
    # print(mask)
    LIB.loc[mask, 'author'] = key
LIB.head()

book_length  n_chaps                  series  year  \
book_title                                                                 
01_a_game_of_thrones       297169       73                  ASOIAF  1996   
01_into_the_wild            64386       26            Warrior_Cats  2003   
01_the_fire_within          51223       42  Last_Dragon_Chronicles  2001   
02_a_clash_of_kings        324987       70                  ASOIAF  1998   
02_fire_and_ice             73544       31            Warrior_Cats  2003   

                                  author  
book_title                                
01_a_game_of_thrones  George R.R. Martin  
01_into_the_wild               Kate Cary  
01_the_fire_within         Chris D'Lacey  
02_a_clash_of_kings   George R.R. Martin  
02_fire_and_ice                Kate Cary

In [100]:
LIB.book_length.mean()

154975.44444444444

### Making the VOCAB

In [101]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [103]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [104]:
DOC = gather_docs(CORPUS, 1)
# DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))

tfidf_engine = TfidfVectorizer(
    stop_words = 'english',
    ngram_range = (1,3),
    # max_features = n_terms,
    norm = 'l2', 
    use_idf = True)

X = tfidf_engine.fit_transform(DOC.doc_str)

DTM = pd.DataFrame(X.toarray(), 
                   columns=tfidf_engine.get_feature_names_out(), 
                   index=DOC.index)

TFIDF = pd.DataFrame(X.toarray(), 
                     columns=tfidf_engine.get_feature_names_out(), 
                     index=DTM.index)

In [105]:
from tqdm import tqdm
import time  # Just for simulating a delay

In [106]:
def get_nltk_pos_tags(n_grams):
    tags = {}
    for n_gram in tqdm(n_grams, desc="Stemming n-grams"):
        tokens = nltk.word_tokenize(n_gram)  # Tokenize the n-gram
        tagged = nltk.pos_tag(tokens)  # Get POS tags
        # Joining tags to match your desired output format
        tags[n_gram] = ' '.join([tag for _, tag in tagged])
    return tags

In [107]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stem_ngrams(n_grams):
    stemmed_ngrams = []
    for n_gram in tqdm(n_grams, desc="Stemming n-grams"):
        tokens = nltk.word_tokenize(n_gram)  # Tokenize the n-gram into individual words
        stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem each word
        stemmed_ngram = ' '.join(stemmed_tokens)  # Rejoin the stemmed words into an n-gram
        stemmed_ngrams.append(stemmed_ngram)
    return stemmed_ngrams

In [108]:
VOCAB = TFIDF[TFIDF > 0].count().to_frame('df')
VOCAB.index.name = 'term_str'
VOCAB['dfidf'] = VOCAB.df * np.log2(len(TFIDF)/VOCAB.df)
VOCAB['df'].fillna(0, inplace=True)
VOCAB['dfidf'].fillna(0, inplace=True)

In [109]:
VOCAB['n'] = VOCAB.index.value_counts()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)

In [110]:
test = list(VOCAB.index)[:10]
# get_nltk_pos_tags(test)
# stem_ngrams(test)

In [111]:
VOCAB['pos_tag'] = get_nltk_pos_tags(list(VOCAB.index))
VOCAB['max_pos'] = VOCAB.groupby(level=0)['pos_tag'].agg(lambda x: x.mode()[0])
VOCAB['pos_group'] = VOCAB.max_pos.str[:2]
VOCAB['max_pos_group'] = VOCAB.groupby(level=0)['pos_group'].value_counts().unstack(fill_value=0).idxmax(1)
# VOCAB.head()

Stemming n-grams: 100%|████████████| 2086449/2086449 [32:13<00:00, 1078.91it/s]


In [112]:
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

VOCAB['stem_porter'] = stem_ngrams(list(VOCAB.index))

# VOCAB[VOCAB.stop == 1].sample(5)

VOCAB.sort_values('dfidf', ascending=False).head(20)

Stemming n-grams: 100%|████████████| 2086449/2086449 [06:59<00:00, 4968.01it/s]


df     dfidf  n             p          i  pos_tag  max_pos  \
term_str                                                                     
effect           7  9.537991  1  4.792832e-07  20.992618       NN       NN   
forests          7  9.537991  1  4.792832e-07  20.992618      NNS      NNS   
claws raked      7  9.537991  1  4.792832e-07  20.992618  NNS VBD  NNS VBD   
lurked           7  9.537991  1  4.792832e-07  20.992618      VBN      VBN   
wasn listening   7  9.537991  1  4.792832e-07  20.992618   NN VBG   NN VBG   
wasn long        7  9.537991  1  4.792832e-07  20.992618    NN RB    NN RB   
nodded slowly    7  9.537991  1  4.792832e-07  20.992618   VBN RB   VBN RB   
kindled          7  9.537991  1  4.792832e-07  20.992618      VBN      VBN   
exquisite        7  9.537991  1  4.792832e-07  20.992618       NN       NN   
think asked      7  9.537991  1  4.792832e-07  20.992618   NN VBD   NN VBD   
whispered ear    7  9.537991  1  4.792832e-07  20.992618   VBN NN   VBN NN   
outlaws          7  9.537991  1  4.792832e-07  20.992618      NNS      NNS   
beetle           7  9.537991  1  4.792832e-07  20.992618       NN       NN   
cat thought      7  9.537991  1  4.792832e-07  20.992618   NN VBD   NN VBD   
borrowed         7  9.537991  1  4.792832e-07  20.992618      VBN      VBN   
did wish         7  9.537991  1  4.792832e-07  20.992618   VBD VB   VBD VB   
dead eyes        7  9.537991  1  4.792832e-07  20.992618   JJ NNS   JJ NNS   
expressionless   7  9.537991  1  4.792832e-07  20.992618       NN       NN   
puckered         7  9.537991  1  4.792832e-07  20.992618      VBN      VBN   
say wanted       7  9.537991  1  4.792832e-07  20.992618   NN VBD   NN VBD   

               pos_group max_pos_group  stop     stem_porter  
term_str                                                      
effect                NN            NN     0          effect  
forests               NN            NN     0          forest  
claws raked           NN            NN     0       claw rake  
lurked                VB            VB     0            lurk  
wasn listening        NN            NN     0     wasn listen  
wasn long             NN            NN     0       wasn long  
nodded slowly         VB            VB     0      nod slowli  
kindled               VB            VB     0           kindl  
exquisite             NN            NN     0        exquisit  
think asked           NN            NN     0       think ask  
whispered ear         VB            VB     0     whisper ear  
outlaws               NN            NN     0          outlaw  
beetle                NN            NN     0           beetl  
cat thought           NN            NN     0     cat thought  
borrowed              VB            VB     0          borrow  
did wish              VB            VB     0        did wish  
dead eyes             JJ            JJ     0        dead eye  
expressionless        NN            NN     0  expressionless  
puckered              VB            VB     0          pucker  
say wanted            NN            NN     0        say want

In [113]:
CORPUS.to_csv(f"{output_dir}\\CORPUS.csv")
LIB.to_csv(f"{output_dir}\\LIB.csv")
# VOCAB.to_csv(f"{output_dir}\\VOCAB.csv")

In [114]:
VOCAB['ngram_length'] = VOCAB.index.str.split().str.len()
VOCAB = VOCAB[VOCAB.ngram_length.isna()==False]

In [115]:
VOCAB.ngram_length.isna().sum()

0

In [116]:
VOCAB.ngram_length = VOCAB.ngram_length.astype(int)

In [117]:
top_20 = VOCAB.sort_values('dfidf', ascending=False).head(20)[["dfidf"]]

In [118]:
VOCAB.to_csv(f"{output_dir}\\VOCAB.csv")

In [119]:
top_20.to_csv(f"{output_dir}\\top_20_dfidf.csv")

In [120]:
top_20

dfidf
term_str                
effect          9.537991
forests         9.537991
claws raked     9.537991
lurked          9.537991
wasn listening  9.537991
wasn long       9.537991
nodded slowly   9.537991
kindled         9.537991
exquisite       9.537991
think asked     9.537991
whispered ear   9.537991
outlaws         9.537991
beetle          9.537991
cat thought     9.537991
borrowed        9.537991
did wish        9.537991
dead eyes       9.537991
expressionless  9.537991
puckered        9.537991
say wanted      9.537991